<h1 style="text-align: center;">Cython_2D particles</h1>

# Python code

In [19]:
import random

import math

import matplotlib.pyplot as plt

import numpy as np

import time

import timeit


# main function
def simulate_py(n, time_step, show_plot=False):

    # random particles
    np.random.seed(0)
    
    p = np.random.rand(n, 2)
    
# Reducing 2D to 1D    p = set_second_column_to_zero(p)

    print("P(0):\n", p)

    # time step loop
    for i in range(time_step):
        
#        print("time step", i)
        
        # calculate total force for each particle
        total_force = combined_force(p, n)
        
#        print("total_force \n",total_force)
        
        # calculate displacement for each particle
        x_det = displacement(total_force, delta_t=0.0001)
        
        #print("x_det \n",x_det)

        # update the particles
        p = update_position(p, x_det)
        
        pos = p
        
        #print("pos \n",pos)
        
        # plot particles
        colors = ['red', 'green', 'blue', 'orange'] 
        if show_plot:

            if i % 2 == 0:

                update_plot(pos,colors)

                print("in iteration", i)
            
    # plot finally result
    print("P({}): ".format(time_step), p)
    
    return p



# Reducing 2 dimensions to 1 dimension for model checking
def set_second_column_to_zero(matrix):

    matrix[:, 1] = 0
    
    return matrix


# Calculate the strength of the repulsion
def f(r, c1=1, c2=1):

    # Calculate the force

    mag = c1 / np.linalg.norm(r)

    #print("mag",mag)

    return mag


# Calculate the total force for each particle
def combined_force(p, n):
    
    total_force = np.zeros_like(p)
    
    for i in range(n):
        
        fn_sum = np.zeros(2)
        
        for j in range(n):
            
            if j != i:
                
                r = p[j] - p[i]
                
                #print("r",r)
                #print("sign",np.sign(r))
                
                fn =  -1 * f(r) * np.sign(r) 

                fn_sum += fn 
                
            total_force[i] = fn_sum
            
    return total_force



# Calculate the displacement between two particles
def displacement(total_force, eta=1, delta_t=1):

    displacement = total_force / eta * delta_t

    return displacement



# Update the position of particles
def update_position(p, delta_r, min_x=0, max_x=1):
    
    new_pos = p + delta_r
    
    x_out_of_bounds = np.logical_or(new_pos[:,0] > max_x, new_pos[:,0] < min_x)
    
    y_out_of_bounds = np.logical_or(new_pos[:,1] > max_x, new_pos[:,1] < min_x)
    
    new_pos[x_out_of_bounds, 0] = np.clip(new_pos[x_out_of_bounds, 0], min_x, max_x)
    
    new_pos[y_out_of_bounds, 1] = np.clip(new_pos[y_out_of_bounds, 1], min_x, max_x)
    
    return new_pos


# Plot
def update_plot(pos,color):

    plt.clf()

    xpos = pos[:, 0]
    
    ypos = pos[:, 1]

    N = len(pos)
    N_color = len(color)
    for i in range(N):
        plt.plot(xpos[i], ypos[i], "o", color=color[i % N_color])

    plt.xlim(left=-0.1, right=1.1)
    
    plt.ylim(bottom=-0.1, top=1.1)

    plt.grid()

    plt.draw()

    plt.pause(0.0001)



# Example usage:
# colors = [random.choice(['r', 'g', 'b', 'y', 'm']) for _ in range(n)]
 
    
#p = simulate_py(n=10, time_step=1000, show_plot=True)

# %prun simulate_py(n=10, time_step=1000, show_plot=False)
compute_time_py = timeit.timeit(lambda: simulate_py(n=10, time_step=1000, show_plot=False), number=1)

print("simulate_py execution time:", compute_time_py)

P(0):
 [[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]
 [0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]
 [0.07103606 0.0871293 ]
 [0.0202184  0.83261985]
 [0.77815675 0.87001215]]
P(1000):  [[0.41916695 0.66633421]
 [0.7845857  0.29828616]
 [0.         0.36688618]
 [0.17622127 1.        ]
 [1.         0.        ]
 [1.         0.12007927]
 [0.72441852 1.        ]
 [0.         0.        ]
 [0.         0.93898401]
 [1.         1.        ]]
simulate_py execution time: 1.06936289999976


# Cython code

In [2]:
%load_ext cython

In [28]:
%%cython --force -c=/openmp -a
#%%cython -a

import numpy as np

cimport numpy as np

from cpython cimport array

import array

import cython

import random

import math

import matplotlib.pyplot as plt

import time

import timeit

from cython.parallel import prange


# random particles
cdef initial_particles(int n):
    
    np.random.seed(0)
    
    cdef np.ndarray[np.float64_t, ndim=2] p
    p = np.random.rand(n, 2)
    
    return p


cimport cython
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef simulate_cy(int n, int time_step, show_plot=False):
    
    cdef np.ndarray[np.float64_t, ndim=2] p

    p = initial_particles(n)
    print("initial_P:\n",p)

    cdef int i
    cdef np.ndarray[np.float64_t, ndim=2] total_force, x_det, pos
    
    for i in range(time_step):
        
        total_force = combined_force(p, n)
    
        x_det = displacement(total_force, delta_t=0.0001)
        
        # update the particles
        p = update_position(p, x_det)
        
        pos = p
        
        # plot particles
        colors = ['red', 'green', 'blue', 'orange'] 
        if show_plot:

            if i % 2 == 0:

                update_plot(pos,colors)

    print("P({}): \n".format(time_step), p)
    
    return p


# Calculate the strength of the repulsion
cdef f(np.ndarray[np.float64_t, ndim=1]r, double c1=1, double c2=1):

    # Calculate the force

    cdef double mag
    
    mag = c1 / math.sqrt(square_sum(r)) 
    
    return mag



cimport cython
@cython.boundscheck(False)
@cython.wraparound(False)

#calculate square
cdef square_sum(np.ndarray[np.float64_t, ndim=1] arr):
    cdef Py_ssize_t i, n
    cdef double s = 0
    n = arr.shape[0]
    for i in prange(n, nogil=True, num_threads=1):
        s += arr[i] * arr[i]
#    print(type(s))
    return s


@cython.boundscheck(False)
@cython.wraparound(False)
# Calculate the total force for each particle
cdef combined_force(np.ndarray[np.float64_t, ndim=2] p, int n):
    
    cdef np.ndarray[np.float64_t, ndim=2] total_force
    
    total_force= np.zeros_like(p,dtype=np.double)
    
    cdef Py_ssize_t i,j
    
    cdef np.ndarray[np.float64_t, ndim=1] r, fn_sum, fn
    
    for i in range(n):
        
        fn_sum = np.zeros(2,dtype=np.double)
        
        for j in range(n):
            
            if j != i:
                
                r = p[j] - p[i]
                
                fn =  -1 * f(r) * np.sign(r) 

                fn_sum += fn 
                
            total_force[i] = fn_sum
            
    return total_force



# Calculate the displacement between two particles
cpdef displacement(np.ndarray[np.float64_t, ndim=2] total_force, double eta=1, double delta_t=1):
    cdef np.ndarray[np.float64_t, ndim=2] displacement

    displacement = total_force / eta * delta_t

    return displacement


cimport cython
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
# Update the position of particles
cdef update_position(np.ndarray[np.float64_t, ndim=2] p, np.ndarray[np.float64_t, ndim=2] delta_r, double min_x=0, double max_x=1):
    
    cdef np.ndarray[np.float64_t, ndim=2] new_pos
    
    new_pos = p + delta_r
    
    x_out_of_bounds = np.logical_or(new_pos[:,0] > max_x, new_pos[:,0] < min_x)
    
    y_out_of_bounds = np.logical_or(new_pos[:,1] > max_x, new_pos[:,1] < min_x)
    

    for i in range(len(new_pos)):
        if x_out_of_bounds[i]:
            new_pos[i, 0] = clip(new_pos[i, 0], min_x, max_x)
        if y_out_of_bounds[i]:
            new_pos[i, 1] = clip(new_pos[i, 1], min_x, max_x)
    
    return new_pos


cdef double clip(a, min_value, max_value):
    return min(max(a, min_value), max_value)


# Plot
cdef update_plot(pos,color):

    plt.clf()

    xpos = pos[:, 0]
    
    ypos = pos[:, 1]

    N = len(pos)
    N_color = len(color)
    for i in range(N):
        plt.plot(xpos[i], ypos[i], "o", color=color[i % N_color])

    plt.xlim(left=-0.1, right=1.1)
    
    plt.ylim(bottom=-0.1, top=1.1)

    plt.grid()

    plt.draw()

    plt.pause(0.0001)

compute_time_cy = timeit.timeit(lambda: simulate_cy(n=10, time_step=1000, show_plot=False), number=1)

print("simulate_cy execution time:", compute_time_cy)

initial_P:
 [[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]
 [0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]
 [0.07103606 0.0871293 ]
 [0.0202184  0.83261985]
 [0.77815675 0.87001215]]
P(1000): 
 [[0.41916695 0.66633421]
 [0.7845857  0.29828616]
 [0.         0.36688618]
 [0.17622127 1.        ]
 [1.         0.        ]
 [1.         0.12007927]
 [0.72441852 1.        ]
 [0.         0.        ]
 [0.         0.93898401]
 [1.         1.        ]]
simulate_cy execution time: 0.42031910000150674


# Compare

In [22]:
from IPython.display import HTML
import pandas as pd

data = {
    'Methods': [ 'Python code','Cython code'],
    'Speed(s)': [compute_time_py, compute_time_cy],
    'Percentage(%)': [100, compute_time_py/compute_time_cy*100]
}
df = pd.DataFrame(data)

# Creating style functions
def add_border(val):
    return 'border: 1px solid black'

# Applying style functions to data boxes
styled_df = df.style.applymap(add_border)

# Defining CSS styles
table_style = [
    {'selector': 'table', 'props': [('border-collapse', 'collapse')]},
    {'selector': 'th, td', 'props': [('border', '1px solid black')]}
]

# Adding styles to stylised data boxes
styled_df.set_table_styles(table_style)

# Displaying stylised data boxes in Jupyter Notebook
HTML(styled_df.to_html())

,Methods,Speed(s),Percentage(%)
0,Python code,1.069363,100.000000
1,Cython code,0.426124,250.951049


## 1. random particles

In [ ]:
def random(n):
    
    np.random.seed(0)
    
    p = np.random.rand(n, 2)
    
    return p

## 2. main function

### 2.1. add types
Since all the functions introduced in the function are of numpy type, the arrays here are defined as np.ndarray[np.float64_t, ndim=2], which represents a 2-dimensional array with elements of np.float64_t.
### 2.2. boundscheck and wraparound
There is a for loop, so close boundscheck and wraparound.
### Note:
np.ndarray[np.float64_t, ndim=2] is a Cython type for numpy arrays. Its definition contains information about the data type and the dimension of the array, allowing functions and methods from the numpy library to be conveniently called. Arrays of type np.ndarray can be manipulated using a variety of numpy methods and functions, providing a high degree of flexibility and versatility.

In contrast, double[:, :] is a Cython-specific type that represents only a two-dimensional array, with no information about the specified data type. Using arrays of type double[:, :] may not allow you to use all numpy functions and methods due to the lack of information about the data type, but its access is faster and can improve the execution of your code to some extent.

Therefore, arrays of type np.ndarray should be used if functions and methods from the numpy library need to be called, or if numerical calculations need to be performed, while arrays of type double[:, :] may be considered if higher execution efficiency is sought.

However,

In Cython, for multi-dimensional arrays, you must use the NumPy array type np.ndarray to perform operations such as division, otherwise an error will occur

In [ ]:
cimport cython
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cpdef simulate_cy(int n, int time_step, show_plot=False):
    
    cdef np.ndarray[np.float64_t, ndim=2] p

    p = random(n)
    print("initial_P:\n",p)

    cdef int i
    cdef np.ndarray[np.float64_t, ndim=2] total_force, x_det, pos
    
    for i in range(time_step):
        
        total_force = combined_force(p, n)
    
        x_det = displacement(total_force, delta_t=0.0001)
        
        # update the particles
        p = update_position(p, x_det)
        
        pos = p
        
        # plot particles
        colors = ['red', 'green', 'blue', 'orange'] 
        if show_plot:

            if i % 2 == 0:

                update_plot(pos,colors)

#                print("in iteration", i)
            
    # plot finally result
    print("P({}): \n".format(time_step), p)
    
    return p

## 3. Calculate the strength of the repulsion
### 3.1 add types
Since all the functions introduced in the function are of numpy type, the arrays here are defined as np.ndarray[np.float64_t, ndim=2], which represents a 2-dimensional array with elements of np.float64_t.
### 3.2 change np.linalg.norm(r)
Replace the original np.linalg.norm(r) with math.sqrt(square_sum(r)) because np.linalg.norm(r) contains the for loop
### 3.3. boundscheck and wraparound
There is a for loop, so close boundscheck and wraparound.
### 3.4 Using multiple threads
from cython.parallel import prange

In [ ]:
# Calculate the strength of the repulsion
cpdef f(np.ndarray[np.float64_t, ndim=1]r, double c1=1, double c2=1):

    # Calculate the force
    
    cdef np.ndarray[np.float64_t, ndim=1] abs_r
    
    abs_r = abs(r)
    
    cdef double mag
    
    mag = c1 / math.sqrt(square_sum(r)) 

    return mag


@cython.boundscheck(False)
@cython.wraparound(False)
#calculate square
cpdef square_sum(np.ndarray[np.float64_t, ndim=1] arr):
    cdef Py_ssize_t i, n
    cdef double s = 0
    n = arr.shape[0]
    for i in prange(n, nogil=True, num_threads=1):
        s += arr[i] * arr[i]
#    print(type(s))
    return s

## 4. Calculate the total force for each particle
### 4.1. add types
Since all the functions introduced in the function are of numpy type, the arrays here are defined as np.ndarray[np.float64_t, ndim=2], which represents a 2-dimensional array with elements of np.float64_t.

For i, j ...... in a for loop, we define them to be of type Py_ssize_t

### 4.2. change np.zeros_like()
### 4.3. boundscheck and wraparound
There is a for loop, so close boundscheck and wraparound.

### 4.4. can not use multiple threads
Because of the repeated calls to numpy in the for loop, gil is needed here and you cannot use multiple threads here

In [ ]:
cimport cython
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
# Calculate the total force for each particle
cdef combined_force(np.ndarray[np.float64_t, ndim=2] p, n):
    
    cdef np.ndarray[np.float64_t, ndim=2] total_force
    
#    total_force = np.zeros_like(p)
    total_force = np.empty_like(p)

    cdef Py_ssize_t k, l
    for k in prange(total_force.shape[0], nogil=True, num_threads=4):
        for l in range(total_force.shape[1]):
            total_force[k, l] = 0
    
    cdef np.ndarray[np.float64_t, ndim=1] fn_sum
    
    cdef np.ndarray[np.float64_t, ndim=1] r
    
    cdef np.ndarray[np.float64_t, ndim=1] fn
 
    
    cdef Py_ssize_t i, j
    
    for i in range(n):
        
        fn_sum = np.zeros(2)
        
        for j in range(n):
            
            if j != i:
                
                r = p[j] - p[i]

                fn =  -1 * f(r) * np.sign(r) 

                fn_sum += fn
           
            total_force[i] = fn_sum
            
    return total_force

## 5. Calculate the displacement between two particles
### 5.1. add types
Since all the functions introduced in the function are of numpy type, the arrays here are defined as np.ndarray[np.float64_t, ndim=2], which represents a 2-dimensional array with elements of np.float64_t.

### 5.2. can not use memoryviews
Because there is division here, it is not possible to compute double[:, :] / double[:, :] in cython, but only as a numpy type

In [ ]:
# Calculate the displacement between two particles
cpdef displacement(np.ndarray[np.float64_t, ndim=2] total_force, double eta=1, double delta_t=1):
    cdef np.ndarray[np.float64_t, ndim=2] displacement

    displacement = total_force / eta * delta_t

    return displacement

# 6. Update the position of particles
## 6.1 add types
## 6.2 np.clip
Changr 'new_pos[x_out_of_bounds, 0] = np.clip(new_pos[x_out_of_bounds, 0], min_x, max_x)'
## 6.3 can not use memoryviews
because np.logical_or()

In [ ]:
cimport cython
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
# Update the position of particles
cdef update_position(np.ndarray[np.float64_t, ndim=2] p, np.ndarray[np.float64_t, ndim=2] delta_r, double min_x=0, double max_x=1):
    
    cdef np.ndarray[np.float64_t, ndim=2] new_pos
    
    new_pos = p + delta_r
    
    x_out_of_bounds = np.logical_or(new_pos[:,0] > max_x, new_pos[:,0] < min_x)
    
    y_out_of_bounds = np.logical_or(new_pos[:,1] > max_x, new_pos[:,1] < min_x)
    

    for i in range(len(new_pos)):
        if x_out_of_bounds[i]:
            new_pos[i, 0] = clip(new_pos[i, 0], min_x, max_x)
        if y_out_of_bounds[i]:
            new_pos[i, 1] = clip(new_pos[i, 1], min_x, max_x)
    
    return new_pos


cdef double clip(a, min_value, max_value):
    return min(max(a, min_value), max_value)